In [1]:
from analysis_functions import * 

2025-03-23 16:48:49.668248: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-23 16:48:49.672792: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-23 16:48:49.687653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-23 16:48:49.711708: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-23 16:48:49.719280: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 16:48:50.920330: W tensorflow/compiler/tf2tensorrt/utils/p

In [10]:
suffixes = [f"{i:03d}" for i in range(3)]
print(suffixes)

ids_cell = list(range(39, 40))
print(ids_cell)

['000', '001', '002']
[39]


In [7]:



def process_cell_features(max_suffix=171, cell_ids=None):
    """
    Process cell features with configurable parameters and detailed error handling.
    
    Args:
        max_suffix (int): Maximum suffix number to process (default: 171)
        cell_ids (list): List of cell IDs to process (default: [1, 2, 3])
    """
    if cell_ids is None:
        cell_ids = list(range(1, 3))  # Default to cells 1-3
    
    data_dir = "../../data/"
    features_dir = "../../features/"  # Base directory for features
    
    # Generate suffixes based on input parameter
    suffixes = [f"{i:03d}" for i in range(max_suffix)]
    
    normalize = False
    fs = 25e6           # Sample frequency
    number_peaks = 10   # Number of peaks
    
    ids_sat = [2, 3, 4, 5, 6, 7, 8, 9, 13, 16,
         17, 18, 22, 23, 24, 25, 26, 28, 29, 30,
         33, 36, 38, 39, 40, 42, 43, 44, 46, 48,
         49, 50, 51, 57, 65, 67, 68, 69, 71, 72,
         73, 74, 77, 78, 79, 81, 82, 85, 87, 88,
         89, 90, 92, 93, 94, 96, 99, 103, 104, 107,
         109, 110, 111, 112, 114, 115]
    
    # Ensure the base features directory exists
    os.makedirs(features_dir, exist_ok=True)
    
    # Counter for processed files and errors
    feature_counter = 1
    error_list = []
    
    print(f"Starting processing for {len(cell_ids)} cells: {cell_ids}")
    print(f"Processing {len(suffixes)} suffixes (0 to {max_suffix-1})")
    
    # Create error report file
    error_report_path = os.path.join(features_dir, "error_report.txt")
    
    # Process each suffix
    for sufix in tqdm(suffixes, desc="Processing suffixes"):
        print(f"\nProcessing suffix {sufix}...")
        try:
            # Load samples and data
            samples = load_samples(data_dir, sufix)
            ra_sat, ra_cell = load_data(data_dir, sufix)
            
            # Create DataFrame
            df = pd.DataFrame({
                'ra_sat': ra_sat,
                'ra_cell': ra_cell,
                'samples': list(samples)  # Store signals in lists
            })
            
            # Index for faster lookup
            df.set_index(['ra_sat', 'ra_cell'], inplace=True)
            
            # Process each cell
            for cell in tqdm(cell_ids, desc=f"Processing cells for suffix {sufix}", leave=False):
                # Create cell-specific directory
                cell_dir = os.path.join(features_dir, f"cell_{cell}")
                os.makedirs(cell_dir, exist_ok=True)
                
                print(f"\nProcessing cell {cell}...")
                signals_cell = get_signals(df=df, cell_id=cell)
                
                if signals_cell:
                    # Process signals for this cell
                    sat_count = 0
                    error_count = 0
                    
                    for idx, (sat_id, signal) in enumerate(signals_cell):
                        if sat_id in ids_sat:
                            try:
                                print(f"- index {idx}, sufix: {sufix}, Cell {cell}, Satellite {sat_id} ")
                                
                                # Extract I/Q components
                                i_signal = signal[:, 0]
                                q_signal = signal[:, 1]
                                
                                # Compute features
                                features_mag, features_phase = iq_features_processor_v1(
                                    i_signal, q_signal, 
                                    fs=fs, 
                                    number_peaks=number_peaks, 
                                    normalize=normalize
                                )
                                
                                # Create feature matrix
                                features, metadata = create_feature_matrix(
                                    features_mag, features_phase, 
                                    sat_label=sat_id, 
                                    transmitter_label=cell, 
                                    feature_counter=feature_counter, 
                                    normalize=normalize
                                )
                                
                                feature_counter += 1
                                sat_count += 1
                                
                                # Save files to cell-specific directory
                                feature_filename = f"{metadata['Feature_Matrix_Name']}"
                                np.save(os.path.join(cell_dir, f"{feature_filename}.npy"), features)
                                with open(os.path.join(cell_dir, f"{feature_filename}_metadata.json"), "w") as f:
                                    json.dump(metadata, f, indent=2)
                                    
                            except Exception as e:
                                error_count += 1
                                error_details = {
                                    "suffix": sufix,
                                    "cell_id": cell,
                                    "sat_id": sat_id,
                                    "df_index": idx,
                                    "error": str(e),
                                    "traceback": traceback.format_exc()
                                }
                                error_list.append(error_details)
                                
                                print(f"  ERROR processing signal: Suffix {sufix}, Cell {cell}, Satellite {sat_id}, Index {idx}")
                                print(f"  Error details: {str(e)}")
                                print("  Continuing to next signal...")
                    
                    print(f"  Processed {sat_count} satellites for cell {cell}, encountered {error_count} errors")
                else:
                    print(f"  No signals found for cell {cell}")
            
        except Exception as e:
            print(f"Error processing entire suffix {sufix}: {str(e)}")
            error_details = {
                "suffix": sufix,
                "cell_id": "ALL",
                "sat_id": "ALL",
                "df_index": "N/A",
                "error": str(e),
                "traceback": traceback.format_exc()
            }
            error_list.append(error_details)
            continue
    
    # Write error report
    with open(error_report_path, "w") as error_file:
        error_file.write(f"ERROR REPORT - Generated on {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        error_file.write(f"Total errors encountered: {len(error_list)}\n\n")
        
        if len(error_list) > 0:
            for i, error in enumerate(error_list, 1):
                error_file.write(f"ERROR #{i}\n")
                error_file.write(f"Suffix: {error['suffix']}\n")
                error_file.write(f"Cell ID: {error['cell_id']}\n")
                error_file.write(f"Satellite ID: {error['sat_id']}\n")
                error_file.write(f"DataFrame Index: {error['df_index']}\n")
                error_file.write(f"Error Message: {error['error']}\n")
                error_file.write(f"Traceback:\n{error['traceback']}\n")
                error_file.write("-" * 80 + "\n\n")
    
    print(f"\nProcessing complete:")
    print(f"- Total features generated: {feature_counter-1}")
    print(f"- Total errors encountered: {len(error_list)}")
    print(f"- Error report saved to: {error_report_path}")

if __name__ == "__main__":
    # Example of how to call the function with custom parameters
    max_suffix = 171  # Process suffixes 000 to 170
    cell_ids = list(range(39, 40))  # Process cells 1, 2, 3
    
    start_time = time.time()
    process_cell_features(max_suffix=max_suffix, cell_ids=cell_ids)
    elapsed_time = time.time() - start_time
    print(f"Total execution time: {elapsed_time:.2f} seconds")

Starting processing for 2 cells: [35, 36]
Processing 3 suffixes (0 to 2)


Processing suffixes:   0%|          | 0/3 [00:00<?, ?it/s]


Processing suffix 000...



Processing cell 35...
- index 0, sufix: 000, Cell 35, Satellite 39 
- index 1, sufix: 000, Cell 35, Satellite 39 
  ERROR processing signal: Suffix 000, Cell 35, Satellite 39, Index 1
  Error details: The cutoff frequencies (0.0, 35149.3059974899) must satisfy 0 < f_low < f_high < 12500000.0 Hz.
  Continuing to next signal...
- index 2, sufix: 000, Cell 35, Satellite 39 
- index 3, sufix: 000, Cell 35, Satellite 39 
- index 4, sufix: 000, Cell 35, Satellite 39 
- index 5, sufix: 000, Cell 35, Satellite 39 
- index 6, sufix: 000, Cell 35, Satellite 39 
- index 7, sufix: 000, Cell 35, Satellite 39 
- index 8, sufix: 000, Cell 35, Satellite 77 
- index 9, sufix: 000, Cell 35, Satellite 77 
- index 10, sufix: 000, Cell 35, Satellite 77 
- index 11, sufix: 000, Cell 35, Satellite 77 
- index 12, sufix: 000, Cell 35, Satellite 77 
- index 13, sufix: 000, Cell 35, Satellite 77 
- index 14, sufix: 000, Cell 35, Satellite 77 
- index 15, sufix: 000, Cell 35, Satellite 77 
- index 16, sufix: 00

- index 407, sufix: 000, Cell 35, Satellite 29 
  ERROR processing signal: Suffix 000, Cell 35, Satellite 29, Index 407
  Error details: The cutoff frequencies (0.0, 40906.91030341476) must satisfy 0 < f_low < f_high < 12500000.0 Hz.
  Continuing to next signal...
  Processed 366 satellites for cell 35, encountered 42 errors

Processing cell 36...
- index 0, sufix: 000, Cell 36, Satellite 40 
- index 1, sufix: 000, Cell 36, Satellite 40 
- index 2, sufix: 000, Cell 36, Satellite 40 
  ERROR processing signal: Suffix 000, Cell 36, Satellite 40, Index 2
  Error details: The cutoff frequencies (0.0, 9971.005801243675) must satisfy 0 < f_low < f_high < 12500000.0 Hz.
  Continuing to next signal...
- index 3, sufix: 000, Cell 36, Satellite 74 
  ERROR processing signal: Suffix 000, Cell 36, Satellite 74, Index 3
  Error details: The cutoff frequencies (0.0, 30458.14243438983) must satisfy 0 < f_low < f_high < 12500000.0 Hz.
  Continuing to next signal...
- index 4, sufix: 000, Cell 36, Sate

Processing suffixes:  33%|███▎      | 1/3 [26:39<53:18, 1599.49s/it]

  Processed 252 satellites for cell 36, encountered 37 errors

Processing suffix 001...



Processing cell 35...
- index 0, sufix: 001, Cell 35, Satellite 29 
- index 1, sufix: 001, Cell 35, Satellite 29 
- index 2, sufix: 001, Cell 35, Satellite 29 
- index 3, sufix: 001, Cell 35, Satellite 29 
- index 4, sufix: 001, Cell 35, Satellite 29 
- index 5, sufix: 001, Cell 35, Satellite 29 
  ERROR processing signal: Suffix 001, Cell 35, Satellite 29, Index 5
  Error details: The cutoff frequencies (0.0, 32827.44570384129) must satisfy 0 < f_low < f_high < 12500000.0 Hz.
  Continuing to next signal...
- index 6, sufix: 001, Cell 35, Satellite 29 
- index 7, sufix: 001, Cell 35, Satellite 29 
  ERROR processing signal: Suffix 001, Cell 35, Satellite 29, Index 7
  Error details: The cutoff frequencies (0.0, 3394.820866661144) must satisfy 0 < f_low < f_high < 12500000.0 Hz.
  Continuing to next signal...
- index 8, sufix: 001, Cell 35, Satellite 29 
- index 9, sufix: 001, Cell 35, Satellite 29 
- index 10, sufix: 001, Cell 35, Satellite 29 
- index 11, sufix: 001, Cell 35, Satelli

Processing suffixes:  33%|███▎      | 1/3 [32:30<1:05:01, 1950.83s/it]


KeyboardInterrupt: 

In [ ]:
data = np.load("../../features/features_1.npy")

# Convertir la matriz en un DataFrame para verla en formato de tabla
df = pd.DataFrame(data)

# Cargar los metadatos desde el archivo JSON
with open("../../features/features_1_metadata.json", "r") as file:
    metadata = json.load(file)

# Obtener nombres de filas y columnas
column_names = metadata["Colum_Feature_Names"]
row_names = metadata["Rows"]

# Convertir la matriz en un DataFrame con nombres de filas y columnas
df = pd.DataFrame(data, index=row_names, columns=column_names)
sat = metadata["Satellite_Label"]
tx = metadata["Transmitter_Label"]
# Mostrar las primeras filas para verificar
print(f"satellite:  {sat}, Transmitter: {tx}")
print(df)